In [2]:
import torch
import torchvision
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
import numpy as np

In [3]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(227),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    #transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

train_data = datasets.CIFAR10('data',train=True,download=True,transform=transform)
test_data = datasets.CIFAR10('data',train=False,download=True,transform=transform)

training_dataloader = DataLoader(train_data,batch_size=128,shuffle=True)
testing_dataloader = DataLoader(test_data,batch_size=128,shuffle=True)


Extracting data/cifar-10-python.tar.gz to data
Files already downloaded and verified


In [4]:
#Architecture
class AlexNet(nn.Module):
  def __init__(self):
    super(AlexNet,self).__init__()
    self.flatten = nn.Flatten()
    self.convolve = nn.Sequential(
        nn.Conv2d(3,96,kernel_size=(11,11),stride=4),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=(3,3),stride=2),
        nn.Conv2d(96,256,kernel_size=(5,5),stride=1,padding=2),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=(3,3),stride=2),
        nn.Conv2d(256,384,kernel_size=(3,3),stride=1,padding=1),
        nn.ReLU(),
        nn.Conv2d(384,384,kernel_size=(3,3),stride=1,padding=1),
        nn.ReLU(),
        nn.Conv2d(384,256,kernel_size=(3,3),stride=1,padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=(3,3),stride=2)
    )
    self.classification = nn.Sequential(
        nn.Dropout(p=0.5),
        nn.Linear(9216,4096),
        nn.ReLU(),
        nn.Dropout(p=0.5),
        nn.Linear(4096,4096),
        nn.ReLU(),
        nn.Linear(4096,10)
    )
  def forward(self,x):
    x=self.convolve(x)
    x=self.flatten(x)
    return self.classification(x)



model = AlexNet()
if torch.cuda.is_available():
  model.cuda()

#optimizer = optim.Adam(model.parameters(),lr=0.0005)
optimizer = optim.SGD(model.parameters(),lr=0.01,weight_decay=0.0005,momentum=0.9)
loss_fn = nn.CrossEntropyLoss()

In [5]:
epoch = 10
for i in range(epoch):
  batch=0
  loss1=0
  for x,y in training_dataloader:
    batch+=1
    if torch.cuda.is_available():
      x,y = x.cuda(),y.cuda()
    
    optimizer.zero_grad()
    output = model(x)
    loss = loss_fn(output,y)
    loss.backward()
    optimizer.step()
    loss1 += loss.item()
  loss1/=batch
  print(f"avg loss in {i+1} epoch is {loss1}")


  batch1=0
  loss2=0
  correct =0
  with torch.no_grad():
    for x,y in testing_dataloader:
      if torch.cuda.is_available():
        x,y = x.cuda(),y.cuda()
      batch1+=1
      pred = model(x)
      loss = loss_fn(output,y)
      loss2+=loss.item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    loss2/=batch
    correct/=len(testing_dataloader.dataset)
    print(f"avg loss while training is {loss2} and accuracy is {correct*100}")
    
    



/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


avg loss in 1 epoch is 2.028945151192453


ValueError: ignored